# 34. Geração de Relatório de Auditoria com Human-in-the-Loop (HITL)

## Objetivo
Este notebook demonstra como usar **LangGraph** para criar um fluxo de trabalho que:
1.  **Gera** um texto de auditoria detalhado usando LLM.
2.  **Pausa** a execução para **Aprovação Humana** (Human-in-the-Loop).
3.  **Salva** o arquivo em disco somente após a aprovação ou edição.

**Conceitos Chave:**
- `MemorySaver`: Para persistir o estado do grafo.
- `interrupt_before`: Para criar breakpoints antes de nós críticos.
- `Command(resume=...)`: Para retomar a execução com novos dados.

In [1]:
%%capture
!pip install -q langgraph langchain langchain-openai pydantic python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from typing import Optional, List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

_ = load_dotenv()

## 1. Definindo o Estado e Modelos
Usaremos Pydantic para estruturar o 'Achado de Auditoria'.

In [3]:
class AchadoAuditoria(BaseModel):
    titulo: str = Field(description="Título curto do achado")
    descricao_detalhada: str = Field(description="Texto completo descrevendo a irregularidade")
    recomendacao: str = Field(description="Ação corretiva sugerida")
    risco: str = Field(description="Nível de risco (Alto, Médio, Baixo)")

class AuditState(TypedDict):
    # Input
    tema: str
    
    # Internal
    draft: Optional[AchadoAuditoria]
    feedback: Optional[str]
    
    # Control
    arquivo_salvo: bool

## 2. Definindo os Nós (Agentes)

In [4]:
# Nó 1: Gerador de Rascunho
def node_gerador(state: AuditState):
    print("\n--- [Gerador] Criando rascunho de auditoria ---")
    tema = state['tema']
    feedback = state.get('feedback')
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    prompt_text = "Você é um auditor experiente. Escreva um achado de auditoria sobre: {tema}."
    if feedback:
        prompt_text += f"\nCONSIDERE O SEGUINTE FEEDBACK DO HUMAN: {feedback}"
        print(f"(Refinando com base no feedback: {feedback})")
    
    prompt = ChatPromptTemplate.from_template(prompt_text)
    chain = prompt | llm.with_structured_output(AchadoAuditoria)
    
    draft = chain.invoke({"tema": tema})
    return {"draft": draft}

In [5]:
# Nó 2: Salvador (Persistência em Disco)
def node_salvador(state: AuditState):
    print("\n--- [Salvador] Salvando arquivo em disco ---")
    draft = state['draft']
    
    filename = "relatorio_auditoria_final.txt"
    conteudo = f"""
    RELATÓRIO DE AUDITORIA
    ======================
    TÍTULO: {draft.titulo}
    RISCO: {draft.risco}
    
    DESCRIÇÃO:
    {draft.descricao_detalhada}
    
    RECOMENDAÇÃO:
    {draft.recomendacao}
    """
    
    with open(filename, "w") as f:
        f.write(conteudo)
        
    print(f"Arquivo salvo com sucesso: {filename}")
    return {"arquivo_salvo": True}

## 3. Construindo o Grafo com Checkpointer
Usaremos `interrupt_before=["Salvador"]` para parar antes de salvar.
Isso dá ao humano a chance de inspecionar o `state['draft']` e decidir se aprova ou pede alterações.

In [6]:
builder = StateGraph(AuditState)

builder.add_node("Gerador", node_gerador)
builder.add_node("Salvador", node_salvador)

builder.add_edge(START, "Gerador")
builder.add_edge("Gerador", "Salvador")
builder.add_edge("Salvador", END)

# Configurando Memória para permitir Pausa/Resume
checkpointer = MemorySaver()

# Compilando com interrupção ANTES de salvar
app = builder.compile(
    checkpointer=checkpointer, 
    interrupt_before=["Salvador"]
)

## 4. Execução Passo 1: Gerar Rascunho
O código irá rodar até o nó `Gerador`, e vai parar **antes** de executar o `Salvador`.

In [7]:
# Configurando uma Thread ID para manter a sessão
thread_config = {"configurable": {"thread_id": "auditoria-001"}}

initial_input = {"tema": "Sobrefaturamento em contrato de limpeza urbana"}

# Executando até a interrupção
# O stream vai parar logo após 'Gerador' terminar e antes de 'Salvador' começar.
for event in app.stream(initial_input, thread_config):
    print(event)


--- [Gerador] Criando rascunho de auditoria ---


{'Gerador': {'draft': AchadoAuditoria(titulo='Sobrefaturamento em Contrato de Limpeza Urbana', descricao_detalhada='Durante a auditoria do contrato de limpeza urbana, identificado sobrefaturamento nas notas fiscais apresentadas pela empresa contratada. Observou-se que os valores cobrados por serviços de coleta de resíduos e varrição estão acima dos preços praticados no mercado e também superiores aos valores estabelecidos na planilha orçamentária do contrato. A análise comparativa com contratos similares em outras localidades confirmou que a empresa está cobrando, em média, 30% a mais pelos serviços prestados. Além disso, foi verificada a ausência de documentação comprobatória que justifique os preços elevados, como estudos de viabilidade ou ajustes por variação de custos.', recomendacao='Recomenda-se a realização de uma revisão detalhada dos contratos em vigor e a implementação de um processo de auditoria contínua nas faturas apresentadas pela empresa contratada. Além disso, sugere-se

## 5. Human-in-the-Loop: Revisão
Agora o processo está **pausado**. Podemos inspecionar o estado atual (`draft`) gerado pelo LLM.

In [8]:
state_atual = app.get_state(thread_config)
draft = state_atual.values['draft']

print("=== RASCUNHO GERADO ===")
print(f"Título: {draft.titulo}")
print(f"Risco: {draft.risco}")
print(f"Descrição: {draft.descricao_detalhada[:200]}...") # Exibindo apenas o início

print("\nO sistema está aguardando aprovação para salvar em disco.")

=== RASCUNHO GERADO ===
Título: Sobrefaturamento em Contrato de Limpeza Urbana
Risco: Alto
Descrição: Durante a auditoria do contrato de limpeza urbana, identificado sobrefaturamento nas notas fiscais apresentadas pela empresa contratada. Observou-se que os valores cobrados por serviços de coleta de r...

O sistema está aguardando aprovação para salvar em disco.


## 6. Decisão Humana
Aqui você pode decidir:
1.  **Aprovar**: Continua a execução e salva o arquivo.
2.  **Rejeitar/Editar**: Atualiza o estado e pede para gerar novamente (ou edita manualmente).

In [9]:
# SIMULAÇÃO: O usuário decidiu que o risco deveria ser 'Extremo' e quer salvar.
# Podemos editar o estado diretamente antes de prosseguir.

user_decision = "aprovar_com_edicao" # Opções: 'aprovar', 'aprovar_com_edicao', 'refazer'

if user_decision == "aprovar_com_edicao":
    print("Usuário editando o risco para 'CRÍTICO/EXTREMO'...")
    
    # Atualizando o estado manualmente
    novo_draft = draft.model_copy()
    novo_draft.risco = "CRÍTICO/EXTREMO"
    
    app.update_state(thread_config, {"draft": novo_draft})
    
    print("Estado atualizado. Prosseguindo para o nó 'Salvador'...")
    
    # Retomando a execução (resume = None, pois só queremos seguir o fluxo normal)
    for event in app.stream(None, thread_config):
        print(event)
        
elif user_decision == "refazer":
    # Se quisesse refazer, poderiamos enviar um feedback e direcionar de volta ao Gerador
    # (Requereria ajustar o grafo para ter loops, aqui simplificamos)
    pass
    
else:
    # Apenas prosseguir
    for event in app.stream(None, thread_config):
        print(event)

Usuário editando o risco para 'CRÍTICO/EXTREMO'...
Estado atualizado. Prosseguindo para o nó 'Salvador'...

--- [Salvador] Salvando arquivo em disco ---
Arquivo salvo com sucesso: relatorio_auditoria_final.txt
{'Salvador': {'arquivo_salvo': True}}


In [10]:
# Verificando se o arquivo foi criado
!ls -l relatorio_auditoria_final.txt
!cat relatorio_auditoria_final.txt

-rw-r--r--@ 1 naubergois  staff  1428 Feb  6 09:49 relatorio_auditoria_final.txt



    RELATÓRIO DE AUDITORIA
    TÍTULO: Sobrefaturamento em Contrato de Limpeza Urbana
    RISCO: CRÍTICO/EXTREMO
    
    DESCRIÇÃO:
    Durante a auditoria do contrato de limpeza urbana, identificado sobrefaturamento nas notas fiscais apresentadas pela empresa contratada. Observou-se que os valores cobrados por serviços de coleta de resíduos e varrição estão acima dos preços praticados no mercado e também superiores aos valores estabelecidos na planilha orçamentária do contrato. A análise comparativa com contratos similares em outras localidades confirmou que a empresa está cobrando, em média, 30% a mais pelos serviços prestados. Além disso, foi verificada a ausência de documentação comprobatória que justifique os preços elevados, como estudos de viabilidade ou ajustes por variação de custos.
    
    RECOMENDAÇÃO:
    Recomenda-se a realização de uma revisão detalhada dos contratos em vigor e a implementação de um processo de auditoria contínua nas faturas apresentadas pela empresa 